In [36]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

In [37]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [38]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [39]:
# Pull Kraken BTC price exchange data
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

Loaded BCHARTS/KRAKENUSD from cache


In [40]:
btc_usd_price_kraken.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


In [41]:
# Chart the BTC pricing data
btc_trace = go.Scatter(x=btc_usd_price_kraken.index, y=btc_usd_price_kraken['Weighted Price'])
py.iplot([btc_trace])

In [42]:
# Pull pricing data for 3 more BTC exchanges
exchanges = ['COINBASE','BITSTAMP','ITBIT']

exchange_data = {}

exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


In [43]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

In [44]:
# Merge the BTC price dataseries' into a single dataframe
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

In [45]:
btc_usd_datasets.tail()

,KRAKEN,COINBASE,BITSTAMP,ITBIT
Date,,,,
2018-04-25,9191.712953,9179.481005,9178.499797,9218.070073
2018-04-26,8881.221549,8905.532970,8881.373724,8890.942698
2018-04-27,9193.347600,9185.267599,9199.122843,9248.551669
2018-04-28,9257.586021,9245.765490,9238.002473,NaN
2018-04-29,9367.415953,9359.029243,9354.092334,NaN


In [46]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [49]:
# Plot all of the BTC exchange prices
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

ValueError: 
    Invalid value of type 'builtins.str' received for the 'visible' property of scatter
        Received value: 'visible'

    The 'visible' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            [True, False, 'legendonly']

In [50]:
# Remove "0" values
btc_usd_datasets.replace(0, np.nan, inplace=True)
# Plot the revised dataframe
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

ValueError: 
    Invalid value of type 'builtins.str' received for the 'visible' property of scatter
        Received value: 'visible'

    The 'visible' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            [True, False, 'legendonly']

In [51]:
# Calculate the average BTC price as a new column
btc_usd_datasets['avg_btc_price_usd'] = btc_usd_datasets.mean(axis=1)
# Plot the average BTC price
btc_trace = go.Scatter(x=btc_usd_datasets.index, y=btc_usd_datasets['avg_btc_price_usd'])
py.iplot([btc_trace])

In [52]:
def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

In [53]:
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

In [54]:
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','SC','XMR','XEM']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1420092000.0&end=1533095407.812967&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1420092000.0&end=1533095407.812967&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1420092000.0&end=1533095407.812967&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETC&start=1420092000.0&end=1533095407.812967&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_STR&start=1420092000.0&end=1533095407.812967&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1420092000.0&end=1533095407.812967&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_SC&start=1420092000.0&end=1533095407.812967&period=86

In [55]:
altcoin_data['ETH'].tail()

,close,high,low,open,quoteVolume,volume,weightedAverage
date,,,,,,,
2018-04-26,0.071486,0.071750,0.067945,0.069600,44267.315458,3100.220866,0.070034
2018-04-27,0.072114,0.073500,0.069847,0.071419,36321.487970,2595.728809,0.071465
2018-04-28,0.073110,0.073778,0.071850,0.072114,17901.737197,1308.574362,0.073098
2018-04-29,0.073115,0.073533,0.072180,0.073110,14356.348738,1049.716682,0.073119
2018-04-30,0.073528,0.073592,0.073158,0.073350,920.389097,67.614450,0.073463


In [56]:
# Calculate USD Price as a new column in each altcoin dataframe
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * btc_usd_datasets['avg_btc_price_usd']
# Merge USD price of each altcoin into single dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')
# Add BTC price to the dataframe
combined_df['BTC'] = btc_usd_datasets['avg_btc_price_usd']
# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (USD)', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale='log')

ValueError: 
    Invalid value of type 'builtins.str' received for the 'visible' property of scatter
        Received value: 'visible'

    The 'visible' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            [True, False, 'legendonly']

In [57]:
# Calculate the pearson correlation coefficients for cryptocurrencies in 2016
combined_df_2016 = combined_df[combined_df.index.year == 2016]
combined_df_2016.pct_change().corr(method='pearson')

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
ETH,1.000000,-0.064652,0.085630,-0.181991,0.035093,0.122695,0.169642,0.087216,0.043205,-0.006502
LTC,-0.064652,1.000000,0.053712,-0.131079,0.113523,-0.012194,0.012253,0.129475,0.160667,0.750174
XRP,0.085630,0.053712,1.000000,-0.054095,0.320116,0.088657,0.021098,0.027649,0.101326,0.044161
ETC,-0.181991,-0.131079,-0.054095,1.000000,-0.102654,0.003992,-0.008066,-0.105898,-0.080938,-0.170538
STR,0.035093,0.113523,0.320116,-0.102654,1.000000,0.058083,0.143252,0.027998,0.225132,0.079075
DASH,0.122695,-0.012194,0.088657,0.003992,0.058083,1.000000,0.026602,0.121537,0.014571,-0.014040
SC,0.169642,0.012253,0.021098,-0.008066,0.143252,0.026602,1.000000,0.047910,0.106153,0.035116
XMR,0.087216,0.129475,0.027649,-0.105898,0.027998,0.121537,0.047910,1.000000,0.016438,0.127520
XEM,0.043205,0.160667,0.101326,-0.080938,0.225132,0.014571,0.106153,0.016438,1.000000,0.227674
BTC,-0.006502,0.750174,0.044161,-0.170538,0.079075,-0.014040,0.035116,0.127520,0.227674,1.000000


In [58]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').as_matrix(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [59]:
correlation_heatmap(combined_df_2016.pct_change(), "Cryptocurrency Correlations in 2016")

C:\Users\priya\Anaconda3\envs\cryptocurrency-analysis\lib\site-packages\ipykernel\__main__.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [60]:
combined_df_2017 = combined_df[combined_df.index.year == 2017]
combined_df_2017.pct_change().corr(method='pearson')

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
ETH,1.000000,0.437609,0.212350,0.601437,0.259399,0.506911,0.373078,0.554632,0.399200,0.410771
LTC,0.437609,1.000000,0.323905,0.482062,0.307589,0.340153,0.339144,0.437204,0.379088,0.420645
XRP,0.212350,0.323905,1.000000,0.114780,0.509828,0.091146,0.243872,0.226636,0.268168,0.131469
ETC,0.601437,0.482062,0.114780,1.000000,0.210387,0.387555,0.298406,0.447398,0.321852,0.416562
STR,0.259399,0.307589,0.509828,0.210387,1.000000,0.183038,0.402966,0.327488,0.339502,0.230957
DASH,0.506911,0.340153,0.091146,0.387555,0.183038,1.000000,0.291424,0.498418,0.325968,0.307095
SC,0.373078,0.339144,0.243872,0.298406,0.402966,0.291424,1.000000,0.378644,0.331350,0.325318
XMR,0.554632,0.437204,0.226636,0.447398,0.327488,0.498418,0.378644,1.000000,0.336076,0.409183
XEM,0.399200,0.379088,0.268168,0.321852,0.339502,0.325968,0.331350,0.336076,1.000000,0.329431
BTC,0.410771,0.420645,0.131469,0.416562,0.230957,0.307095,0.325318,0.409183,0.329431,1.000000


In [61]:
correlation_heatmap(combined_df_2017.pct_change(), "Cryptocurrency Correlations in 2017")

C:\Users\priya\Anaconda3\envs\cryptocurrency-analysis\lib\site-packages\ipykernel\__main__.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [62]:
combined_df_2018 = combined_df[combined_df.index.year == 2018]
combined_df_2018.pct_change().corr(method='pearson')

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
ETH,1.000000,0.746078,0.709356,0.814798,0.671741,0.860311,0.643214,0.811110,0.716969,0.812288
LTC,0.746078,1.000000,0.689536,0.712461,0.647435,0.850456,0.681293,0.764954,0.600686,0.835746
XRP,0.709356,0.689536,1.000000,0.681743,0.843919,0.789851,0.701558,0.703001,0.791922,0.749043
ETC,0.814798,0.712461,0.681743,1.000000,0.620708,0.792274,0.586025,0.715201,0.639621,0.731339
STR,0.671741,0.647435,0.843919,0.620708,1.000000,0.738634,0.657962,0.688697,0.768490,0.710181
DASH,0.860311,0.850456,0.789851,0.792274,0.738634,1.000000,0.719253,0.878043,0.725753,0.888735
SC,0.643214,0.681293,0.701558,0.586025,0.657962,0.719253,1.000000,0.630301,0.670062,0.734644
XMR,0.811110,0.764954,0.703001,0.715201,0.688697,0.878043,0.630301,1.000000,0.674740,0.871210
XEM,0.716969,0.600686,0.791922,0.639621,0.768490,0.725753,0.670062,0.674740,1.000000,0.702006
BTC,0.812288,0.835746,0.749043,0.731339,0.710181,0.888735,0.734644,0.871210,0.702006,1.000000


In [63]:
correlation_heatmap(combined_df_2018.pct_change(), "Cryptocurrency Correlations in 2018")

C:\Users\priya\Anaconda3\envs\cryptocurrency-analysis\lib\site-packages\ipykernel\__main__.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [64]:
correlation_heatmap(combined_df.pct_change(), "Overall Cryptocurrency Correlations")

C:\Users\priya\Anaconda3\envs\cryptocurrency-analysis\lib\site-packages\ipykernel\__main__.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [65]:
combined_df

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
date,,,,,,,,,,
2015-01-02,NaN,2.685418,0.024511,NaN,0.005536,1.974292,NaN,0.455994,NaN,315.785146
2015-01-03,NaN,2.389670,0.022297,NaN,0.005117,1.818238,NaN,0.473959,NaN,301.512433
2015-01-04,NaN,2.079496,0.018999,NaN,0.004521,1.705425,NaN,0.424769,NaN,276.706053
2015-01-05,NaN,2.076988,0.020051,NaN,0.004870,1.650610,NaN,0.413943,NaN,271.139494
2015-01-06,NaN,2.091707,0.020350,NaN,0.004928,1.667679,NaN,0.440428,NaN,274.256460
2015-01-07,NaN,2.183424,0.020957,NaN,0.005164,1.742234,NaN,0.462845,NaN,293.551047
2015-01-08,NaN,2.099834,0.020871,NaN,0.005563,1.730026,NaN,0.447558,NaN,290.364003
2015-01-09,NaN,2.051141,0.021352,NaN,0.005792,1.747407,NaN,0.428848,NaN,291.770886
2015-01-10,NaN,1.801965,0.020094,NaN,0.005455,1.711341,NaN,0.403604,NaN,282.503461


In [66]:
combined_df

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
date,,,,,,,,,,
2015-01-02,NaN,2.685418,0.024511,NaN,0.005536,1.974292,NaN,0.455994,NaN,315.785146
2015-01-03,NaN,2.389670,0.022297,NaN,0.005117,1.818238,NaN,0.473959,NaN,301.512433
2015-01-04,NaN,2.079496,0.018999,NaN,0.004521,1.705425,NaN,0.424769,NaN,276.706053
2015-01-05,NaN,2.076988,0.020051,NaN,0.004870,1.650610,NaN,0.413943,NaN,271.139494
2015-01-06,NaN,2.091707,0.020350,NaN,0.004928,1.667679,NaN,0.440428,NaN,274.256460
2015-01-07,NaN,2.183424,0.020957,NaN,0.005164,1.742234,NaN,0.462845,NaN,293.551047
2015-01-08,NaN,2.099834,0.020871,NaN,0.005563,1.730026,NaN,0.447558,NaN,290.364003
2015-01-09,NaN,2.051141,0.021352,NaN,0.005792,1.747407,NaN,0.428848,NaN,291.770886
2015-01-10,NaN,1.801965,0.020094,NaN,0.005455,1.711341,NaN,0.403604,NaN,282.503461


In [67]:
combined_df.to_csv('price.csv')